# Experiments
### Notebook to run experiments on the data supplied by the chair of functional materials at Saarland University

## Experiment 1
### Run RAFT optical flow model on the data
<font size="5"> 
In this experiment we will run the RAFT optical flow model on the data to see how well it performs compared to classical optical flow methods.
</font>

### Results
<font size="5"> 
RAFT tracks the reaction front quite well. The reaction front is visible in the optical flow images but there is also a lot of noise.

</font>

In [1]:
import torch
import numpy as np
import os
from tqdm import tqdm
import cv2
from raft.utils import get_video_frames, plot_frames, preprocess

In [2]:
video_path = "videos/DoubleC-shape/double_c_shape.mp4"
frames = get_video_frames(video_path)

In [ ]:
# Change the numbers in frames to see different images.
img_batch = torch.stack([frames[50], frames[51]])
plot_frames(img_batch)

In [4]:
from torchvision.models.optical_flow import raft_large
from torchvision.models.optical_flow.raft import Raft_Large_Weights

model = raft_large(weights=Raft_Large_Weights.C_T_V2)
model = model.eval()

In [ ]:
img1_batch: torch.Tensor = preprocess(torch.stack([frames[130], frames[220]]))
img2_batch: torch.Tensor = preprocess(torch.stack([frames[131], frames[221]]))

plot_frames(img1_batch)

In [6]:
predicted_flows: torch.Tensor = model(img1_batch, img2_batch)[-1]

In [ ]:
from torchvision.utils import flow_to_image
flow_images = flow_to_image(predicted_flows)

# The images have been mapped into [-1, 1] but for plotting we want them in [0, 1]
img1_batch = (img1_batch + 1.) / 2.

grid = [[img1, flow_img] for (img1, flow_img) in zip(img1_batch, flow_images)]
plot_frames(grid) 

## Experiment 2
### Obtain reaction front from difference between frames
<font size="5"> 
In this experiment we will see how well we can obtain the reaction front by simply taking the difference between frames. We set pixels with very small and large differences between two frames to zero, as they seem to be noise. We also binarize the image to make the front more visible. For further denoising we apply a morphological operation called <i>Opening</i>, which removes all white spots smaller than a certain size. Mathematical morphology in the context of image processing deals with analysing and modifying structures in an image based on their geometrical properties. 
</font>

### Results
<font size="5"> 
The reaction front is actually quite visible in the difference between frames. This is a very simple method to obtain the reaction front.  The method also seems to outperform the old optical flow method in terms of noise and also speed.
</font>

In [2]:
from skimage import morphology
from tqdm import tqdm
from utils.video_handling import get_video_frames
import numpy as np
import cv2
import os

In [6]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
footprint = morphology.disk(3)

for video_name in os.listdir("videos/"): 
    print(f"Generating flow for {video_name}")
    video_path = "videos/" + video_name
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()
    
    frames = get_video_frames(video_path)
    video_writer = cv2.VideoWriter("results/experiment2/" + video_name, fourcc, 10.0, (width, height))
    for i, _ in tqdm(enumerate(frames[:-1])):
        # Calculate the difference between two frames.
        flow = frames[i+1] - frames[i]
        flow = flow.astype(np.uint8)
        
        # Threshold and binarize the image.
        flow[flow > 180] = 0
        flow[flow < 20] = 0
        flow[flow > 0] = 255
        
        # Apply morphological opening.
        flow = morphology.opening(flow, footprint)
        flow = cv2.cvtColor(flow, cv2.COLOR_GRAY2BGR)
        video_writer.write(flow)
        
    video_writer.release()

Generating flow for comb_shape.mp4


198it [00:12, 15.84it/s]


Generating flow for c_shape.mp4


200it [00:12, 15.73it/s]


Generating flow for double_c_shape.mp4


...

190it [00:12, 15.66it/s]


Generating flow for no_shape.mp4


160it [00:10, 15.80it/s]


Generating flow for vertical_lines.mp4


190it [00:12, 15.82it/s]
